In [2]:
import warnings
warnings.filterwarnings("ignore")
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import torch

from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'albedo',
    
    # Thư mục
    'dataset_dir': './Dataset',
    'checkpoint_dir': './checkpoint/multi/',
    
    # Cấu hình train
    'device': device,
    'epochs': 100,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'base_lr': 1e-4,
    
    # Cấu hình network
    'backbone': 'miresnet18',
    'embedding_size': 512,
    'num_classes': None,
    'loss_spectacles_weight': 0.1,
    'loss_da_spectacles_weight': 0.1,
    'loss_occlusion_weight': 0.1,
    'loss_da_occlusion_weight': 0.1,
    'loss_facial_hair_weight': 0.1,
    'loss_da_facial_hair_weight': 0.1,
    'loss_pose_weight': 0.2,
    'loss_da_pose_weight': 0.2,
    'loss_gender_weight': 0.1,
    'loss_da_gender_weight': 0.1,
    'loss_emotion_weight': 0.2,
    'loss_da_emotion_weight': 0.2,
}

CONFIGURATION['num_classes'] = len(os.listdir('./Dataset/Albedo/train'))

train_transform = A.Compose([
    RandomResizedCropRect(256),
    A.Rotate(limit=20, p=0.5),
    GaussianNoise(),
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})

train_dataloader, test_dataloader, train_weight_class = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)
model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])

criterion = MultiTaskLoss(os.path.join(CONFIGURATION['dataset_dir'], 'train_set.csv'), CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
# Khởi tạo scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      
checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

2025-01-14 15:50:03.470875: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 15:50:04.765544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    criterion=criterion,
    optimizer=optimizer,    
    scheduler=scheduler, 
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
╒═════════════════════╤═══════════╤══════════════════════╕
│ Metric              │     Train │ Test                 │
╞═════════════════════╪═══════════╪══════════════════════╡
│ loss                │ 3.51119   │ -                    │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_id             │ 3.2592    │ -                    │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_gender         │ 0.0636336 │ 0.08311351691372693  │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_da_gender      │ 0.0665108 │ 0.09672368981409818  │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_emotion        │ 0.307721  │ 0.3163571376353502   │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_da_emotion     │ 0.377948  │ 0.37897881492972374  │
├─────────────────────┼───────────┼──────────────────────┤
│ loss_pose           │ 0.188404  │ 0.17501253448426723  │
├─────────────────────┼───────────┼────────────

In [3]:
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [4]:
import cv2, os

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

image_path = './Dataset/Albedo/gallery/1002/2008-02-15_16-27-21.exr'

image = cv2.cvtColor(cv2.imread(image_path, cv2.IMREAD_UNCHANGED), cv2.COLOR_GRAY2RGB)

transfromed = test_transform(image=image)

X = torch.from_numpy(transfromed['image']).permute(2,0,1).unsqueeze(0)

In [9]:
model.eval()
x_id, x_gender, x_pose, x_emotion, x_facial_hair, x_occlusion, x_spectacles = model.get_embedding(X)

In [13]:
x_emotion = torch.softmax(x_emotion, dim=1)
x_emotion

tensor([[0.2065, 0.0035, 0.7901]], grad_fn=<SoftmaxBackward0>)